<a href="https://colab.research.google.com/github/viv-bad/pytorch-course/blob/master/03_pytorch_computer_vision_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0.1 Computer vision libraries in PyTorch

* `torchvision` - base/main computer vision library in PyTorch
* `torchvision.datasets` - get datasets and data loading functions for computer vision
* `torchvision.models` - pretrained computer vision models that you can leverage for your own problems
* `torchvision.transforms` - functions for manipulating your vision data (images) to be suitable for use with an ML model.
* `torch.utils.data.Dataset` - to create our own dataset with out own data, we have the base class for this in PyTorch
* `torch.utils.data.DataLoader` - Creates a Python iterable over a dataset.




In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
import torchvision.transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
print (torch.__version__)
print (torchvision.__version__)

# Setup device agnostic code
import torch
device = 'cuda' if torch.cuda.is_available() else "cpu"

device

In [ ]:
### 1. Setup training data
train_data = datasets.FashionMNIST(
    root="data", #where to download data to
    train=True, # do we want the training dataset/testing dataset
    download=True, #do we want to download it
    transform=ToTensor(),
    target_transform=None, #do we want to transform the target data/labels
)

test_data = datasets.FashionMNIST(
    root="data", #where to download data to
    train=False, # do we want the training dataset/testing dataset
    download=True, #do we want to download it
    transform=ToTensor(),
    target_transform=None, #do we want to transform the target data/labels
)




In [ ]:
len(train_data), len(test_data)

In [ ]:
# See the first training example
image, label = train_data[0]
image, label

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx #we have 10 different items of clothing, so output shape must be 10

In [ ]:
train_data.targets

In [ ]:
# Check the shape of our image
print(f"Image shape: {image.shape} -> [color_channels, height, width]")
print(f"Image label: {class_names[label]}")

### 1.2 Visualising our data

In [ ]:
image, label = train_data[0]
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze()) # matplotlib expects [height, width, colour_channel], so squeeze to get rid of colour channel dimension
plt.title(f"{label},  {class_names[label]}")

In [ ]:
plt.imshow(image.squeeze(), cmap="gray")
plt.title(f"{label},  {class_names[label]}")
plt.axis(False)

In [ ]:
# Plot more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols = 4,4
for i in range(1, rows*cols+1):
  random_idx = torch.randint(0, len(train_data), size=[1]).item()
  img, label = train_data[random_idx]
  fig.add_subplot(rows, cols, i)
  plt.imshow(img.squeeze(), cmap="gray")
  plt.title(f"{label},  {class_names[label]}")
  plt.axis(False)

Do you think these items of clothing (images) could be modelled with pure linear lines? Or do we need non linearity

## 2. Prepare DataLoader

Right now, our data is in the form of PyTorch Datasets.

DataLoader turns our dataset into a Python iterable.

More specifically, we want to turn our data into batches (or mini-batches).

Why?

1. It is more computationally efficient, as in your comp hardware may not be able to look (store in memory) all the images in one hit. So break it down to 32 images at one time (batch_size=32).

2. It gives our neural network more chances to update its gradients per epoch.


In [ ]:
# Turn train dataset into DataLoader
from torch.utils.data import DataLoader
BATCH_SIZE = 32
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True) # shuffle training data to remove order, so training data doesn't have order
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)# don't shuffle test/eval data, it's good to have the data in the same order. our model will never actually see the test data during training anyway

train_dataloader, test_dataloader

In [ ]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Length of train_dataloader: {len(train_dataloader)}, batches of {BATCH_SIZE}")
print(f"Length of test_dataloader: {len(test_dataloader)}, batches of {BATCH_SIZE}")

In [ ]:
# Check out what's inside train dataloader
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
# Show a sample
torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(class_names[label])
plt.axis(False)
print(f"Image size: {img.shape}")
print(f"Label: {label}, label size: {label.shape}")

## 3. Model 0: Build a baseline model

When starting to build a series of ML. modelling experiments, always start with a baseline model.

A baseline model is a simple model you will try to improve upon with subsequent models/experiments.

Start simple and add complexity when necessary.

In [ ]:
# Create a flatten layer
flatten_model = nn.Flatten()

# Get a single sample
x = train_features_batch[0]
x.shape

# Flatten the sample
output = flatten_model(x) # perform forward pass

# Print out what happened
print(f"Shape before flattening: {x.shape} -> [colour_channels, height, width]")
print(f"Shape after flattening: {output.shape} -> [colour_channels, height*width]") # now after flattening we have one value per pixel (height*width)


In [ ]:
from torch import nn

class FashionMNISTModelV0(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()

    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.Linear(in_features=hidden_units, out_features=output_shape)
    )

  def forward(self, x):
    return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)

# Set up model with input params

model_0 = FashionMNISTModelV0(input_shape=784, #28x28
                              hidden_units=10,
                              output_shape=len(class_names) #one for every class

                              ).to(device)

model_0

In [ ]:
dummy_x = torch.rand([1,1,28,28]).to(device)
model_0(dummy_x) # output of logits

In [ ]:
model_0.state_dict()

### 3.1 Setup loss, optimizer and evaluation metrics

* Loss function - since we're working with multiclass data, our loss function is `nn.CrossEntropyLoss()`
* Optimizer - we will stick with `torch.optim.SGD()` (stochastic gradient descent)
* Evaluation metric - since we're working on a classification problem, let's use accuracy as our evaluation metric.


In [ ]:
import requests
from pathlib import Path

if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

# Setup loss function and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.1)

### 3.2 Creating a function to time our experiments

Machine learning is very experimental.

Two main things to track are:

1. Model's performance (loss, accuracy etc)
2. How fast it runs




In [ ]:
from timeit import default_timer as timer

def print_train_time(start: float, end: float, device: torch.device = None):
  """
  Prints difference between start and end time"""

  total_time = end-start

  print(f"Train time on {device}: {total_time: .3f} seconds")
  return total_time

In [ ]:
start_time = timer()

end_time = timer()

print_train_time(start=start_time, end=end_time, device=device)


### 3.3 Creating a training loop and training a model on batches of data

1. Loop through epochs
2. Loop through training batches, perform training steps, calculate the train loss *per batch*
3. Loop through testing batches, perform testing steps, calculate test loss *per batch*
4. Print out what's happening
5. Time is all

In [ ]:
# Import tqdm for a progress bar
from tqdm.auto import tqdm

# set seed and start timer
torch.manual_seed(42)
train_time_start_on_cpu = timer()

# Set number of epochs (we'll keep it small for faster training time)
epochs = 3

# Create training and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-----")
  ### Training
  train_loss = 0
  # Add loop to loop thorugh training batches
  for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    model_0.train()

    # 1. Forward pass
    y_pred = model_0(X)

    # 2. Calculate loss (per batch)
    loss = loss_fn(y_pred, y)
    train_loss += loss # accumulate train loss

    # 3. optimizer zero grad
    optimizer.zero_grad()

    # 4.  Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Print progress only every 400 batches
    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  # Divide total train loss by length of train dataloader
  train_loss /= len(train_dataloader)

  ### Testing
  test_loss, test_acc = 0, 0
  model_0.eval()
  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      X_test, y_test = X_test.to(device), y_test.to(device)
      # 1. forward pass - FIXED: Using X_test instead of X
      test_pred = model_0(X_test)

      # 2. Calculate loss accumulatively
      test_loss += loss_fn(test_pred, y_test)

      # 3. Calculate accuracy
      test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

    # Calculate test loss average per batch
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  # Print out
  print(f"\nTrain loss: {train_loss:.4f} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")

# Calculate training time - FIXED: Made device agnostic
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(
    start=train_time_start_on_cpu,
    end=train_time_end_on_cpu,
    device=next(model_0.parameters()).device  # Get the actual device the model is on
)

## 4.  Make predictions and get Model 0 results


In [ ]:
torch.manual_seed(42)

def eval_model(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn, device):
  """Returns a dict containing the results of model predicting on data_loader"""
  loss, acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X, y in data_loader:
      X, y = X.to(device), y.to(device)
      # make predictions
      y_pred = model(X)

      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

    # Scale loss and acc to find average loss/acc per batch
    loss /= len(data_loader)
    acc /= len(data_loader)

  # Handle both tensor and float types appropriately
  result = {"model_name": model.__class__.__name__}

  # Check if loss is a tensor or float
  if isinstance(loss, torch.Tensor):
    result["model_loss"] = loss.item()
  else:
    result["model_loss"] = loss

  # Check if acc is a tensor or float
  if isinstance(acc, torch.Tensor):
    result["model_acc"] = acc.item()
  else:
    result["model_acc"] = acc

  return result


In [ ]:

# Calculate model - results on test dataset
model_0_results = eval_model(model=model_0, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn=accuracy_fn, device=device)

model_0_results

In [ ]:
# Setup device agnostic code
import torch
device = 'cuda' if torch.cuda.is_available() else "cpu"

device

## Modal 1: Building a better model with non-linearity

We learned about the power of non-linearity in notebook 2.

In [ ]:
class FashionMNISTModelV1(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()

    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_shape),
        nn.ReLU()
    )

  def forward(self, x: torch.Tensor):
    return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)

model_1 = FashionMNISTModelV1(input_shape=784, #28x28
                              hidden_units=10,
                              output_shape=len(class_names) #one for every class

                              ).to(device)

next(model_1.parameters()).device

In [ ]:
from helper_functions import accuracy_fn

# Setup loss function and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.1)

### 6.2 Functionizing training and testing loops

In [ ]:
def train_step(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, accuracy_fn, device: torch.device = device):
  """Performs training step with model trying to learn on data_loader"""
  train_loss, train_acc = 0,0
  model.train()
  for batch, (X, y) in enumerate(data_loader):
    # Put data on target device
    X, y = X.to(device), y.to(device)

    y_pred = model(X)

    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy_fn(y_true = y, y_pred = y_pred.argmax(dim=1))

    optimizer.zero_grad()


    loss.backward()


    optimizer.step()

        # Print progress only every 400 batches
    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

  # Divide total train loss by length of train dataloader
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)

  print(f"Train loss: {train_loss : .5f} | Train acc: {train_acc : .2f} %")


In [ ]:
def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device):
    """Performs a testing loop step on model going over data_loader"""

    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # send to device
            X, y = X.to(device), y.to(device)

            # 1. forward pass
            test_pred = model(X)

            # 2. Calculate loss accumulatively
            test_loss += loss_fn(test_pred, y)

            # 3. Calculate accuracy
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        # Calculate test loss average per batch
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)

    # Print out (removing train_loss reference since it's not available here)
    print(f"\nTest loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

    return test_loss, test_acc

In [ ]:
torch.manual_seed(42)

train_time_start_on_gpu = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n--------")
  train_step(model=model_1, data_loader=train_dataloader, loss_fn=loss_fn, optimizer = optimizer, accuracy_fn = accuracy_fn, device = device)
  test_step(model=model_1, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn = accuracy_fn, device = device)

train_time_end_on_gpu = timer()

total_train_time_model_1 = print_train_time(
    start=train_time_start_on_gpu,
    end=train_time_end_on_gpu,
    device=device # Get the actual device the model is on
)



>**Note:** Sometimes, depending on your data/hardware, you might find that your model trains faster on CPU than GPU.
?Why is this?

>1. It could be that the overhead for copying data/model to and from GPU outweighs the compute benefits offered by the GPU.
>2. The hardware you're using has a better CPU in terms of compute capability than the GPU (unlikely).


In [ ]:
model_0_results

In [ ]:
total_train_time_model_0

In [ ]:
# Get model_1 results dictionary
model_1_results = eval_model(model=model_1, data_loader = test_dataloader, loss_fn=loss_fn, accuracy_fn=accuracy_fn, device=device)
model_1_results

## Model 2: Building a Convolutional Neural Network (CNN)

CNN's are also known as ConvNets.

CNN's are known for their capabilities to find patterns in visual data.

More layers = more chance to find patterns in data

In [ ]:
# Create a convolutional neural network
class FashionMNISTModelV2(nn.Module):
  """Model Architecture that replicates the TinyVGG mpdel from the CNN explainer website."""
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2) #takes the max value of the input and outputs it
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels = hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2) #takes the max value of the input and outputs it
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units * 7 * 7, # There's a trick to calculating this
                  out_features=output_shape)
    )

  def forward(self, x):
    x = self.conv_block_1(x)
    # print(f"Output shape of conv_block_1: {x.shape}")
    x = self.conv_block_2(x)
    # print(f"Output shape of conv_block_2: {x.shape}")
    x = self.classifier(x)
    # print(f"Output shape of classifier: {x.shape}")
    return x

In [ ]:
torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape=1, #black and white image, so only one colour channel, colour = 3
                              hidden_units=10,
                              output_shape=len(class_names)
                              ).to(device)

model_2

## 7.1 Stepping through `nnConv2d()`

https://poloclub.github.io/cnn-explainer/

Conv2D (Convolution 2D)
Imagine you have an image represented as a grid of numbers (pixel values). A Conv2D operation works by:

Using a filter (or kernel): This is a small grid of numbers (e.g., 3×3 or 5×5).
Sliding this filter: The filter moves across the original image grid, step by step.
Performing multiplication: At each position, the filter is placed over a section of the image, and each filter value is multiplied with the corresponding image value underneath it.
Summing up: All these multiplications are added together to produce a single number.
Building a new image: As the filter slides across the entire image, it creates a new grid of numbers (the "feature map").

Think of it like shining a flashlight over different parts of an image, where the flashlight highlights certain features (like edges, textures, or shapes) depending on what the filter is designed to detect.

In [ ]:
torch.manual_seed(42)

# Create a batch of images
images = torch.randn(size=(32, 3, 64, 64))
test_image = images[0]

print(f"Image batch shape: {images.shape}")
print(f"Single image shape: {test_image.shape}")
print(f"Test image: \n{test_image}")

In [ ]:
torch.manual_seed(42)
# Create a single conv2d layer
conv_layer = nn.Conv2d(in_channels=3, # in_channels is the same number of colour channels as the colour input image
                       out_channels=10, #equial to the number of hidden layers
                       kernel_size=3,
                       stride=1,
                       padding=1
                       )

# Pass the data through the convolutional layer
conv_output = conv_layer(test_image)
conv_output.shape

### 7.2 Stepping through `nn.MaxPool2d()`

MaxPool2D (Max Pooling 2D)
After applying convolutions, MaxPool2D helps reduce the size of the feature maps:

Defining a window: Usually a small 2×2 grid.
Sliding this window: Moving it across the feature map (typically without overlap).
Taking the maximum: In each window position, only the largest value is kept.
Creating a smaller output: The result is a downsized version of the original feature map.

MaxPooling essentially says, "In this small region, let's just keep the strongest signal and ignore the rest." This:

Reduces the computational load
Makes the network focus on the most important features
Helps the network become less sensitive to small shifts in the input image

Together, Conv2D layers detect patterns while MaxPool2D layers summarize them and make the network more efficient.

In [ ]:
# plt.imshow(image.to(device).squeeze(), cmap="gray")

In [ ]:
# Pass image through model
rand_image_tensor = torch.randn(size=(1,28,28)).to(device)
rand_image_tensor

In [ ]:
model_2(rand_image_tensor.unsqueeze(0))

In [ ]:
# Print out image shape without unsqueeze (no need for this anyway it works fine in this version of pytorch)
print(f"Test image original shape: {test_image.shape}")
print(f"Test image with unsqueezed dimension: {test_image.unsqueeze(0).shape}" )

# Create a sample nn.MaxPool2d layer
max_pool_layer = nn.MaxPool2d(kernel_size=2)

# Pass data through just the conv_layer
test_image_through_conv = conv_layer(test_image.unsqueeze(0))
print(f"Shape after going through conv_layer(): {test_image_through_conv.shape}")

# Pass data through the max pool layer
test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
print(f"Shape after going thorugh conv_layer() and max_pool_layer(): {test_image_through_conv_and_max_pool.shape}")


In [ ]:
torch.manual_seed(42)

# Create random tensor with a similar number of dimensions to our images
random_tensor = torch.randn(size=(1,1,2,2))
print(f"\n Random tensor: \n{random_tensor}")
print(f"\n Random tensor shape: \n{random_tensor.shape}")
# Create a max pool layer
max_pool_layer = nn.MaxPool2d(kernel_size=2)

# Pass random tensor through maxpool layer
max_pool_tensor = max_pool_layer(random_tensor)
print(f"\n Max pool tensor: {max_pool_tensor}")
print(f"Max pool tensor shape: {max_pool_tensor.shape}")


### 7.3 Seup a loss function and optimizer for `model_2`

In [ ]:
# Set up loss function/eval metrics/optimizer
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

train_time_start_model_2 = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n--------")
  train_step(model=model_2, data_loader=train_dataloader, loss_fn=loss_fn, optimizer = optimizer, accuracy_fn = accuracy_fn, device = device)
  test_step(model=model_2, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn = accuracy_fn, device = device)

train_time_end_model_2 = timer()

total_train_time_model_2 = print_train_time(
    start=train_time_start_on_gpu,
    end=train_time_end_on_gpu,
    device=device # Get the actual device the model is on
)



In [ ]:
# Get model_2 results
model_2_results = eval_model(
    model=model_2, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn=accuracy_fn, device=device
)
model_2_results

## 8. Compare model results and training time

In [ ]:
import pandas as pd
compare_results = pd.DataFrame([model_0_results, model_1_results, model_2_results])

compare_results

### Improve model

In [ ]:
class ImprovedFashionMNISTModel(nn.Module):
    """Improved model architecture for FashionMNIST dataset with batch normalization,
    dropout, and residual connections."""
    def __init__(self, input_shape: int = 1, hidden_units: int = 64, output_shape: int = 10):
        super().__init__()

        # First convolutional block with batch normalization
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # Second convolutional block with increased channels
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units*2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_units*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units*2, out_channels=hidden_units*2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_units*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # Third convolutional block for more depth
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units*2, out_channels=hidden_units*2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_units*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units*2, out_channels=hidden_units*4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_units*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # Global average pooling and classifier with dropout
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),  # Global average pooling
            nn.Flatten(),
            nn.Dropout(0.3),  # Adding dropout for regularization
            nn.Linear(in_features=hidden_units*4, out_features=hidden_units*2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(in_features=hidden_units*2, out_features=output_shape)
        )

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.classifier(x)
        return x

In [ ]:
# def train_step(model, data_loader, loss_fn, optimizer, accuracy_fn, device):
#     """Performs a training step for a PyTorch model."""
#     # Put model in train mode
#     model.train()

#     # Setup train loss and train accuracy values
#     train_loss, train_acc = 0, 0

#     # Loop through data loader batches
#     for batch, (X, y) in enumerate(data_loader):
#         # Send data to target device
#         X, y = X.to(device), y.to(device)

#         # 1. Forward pass
#         y_pred = model(X)

#         # 2. Calculate loss
#         loss = loss_fn(y_pred, y)
#         train_loss += loss.item()

#         # 3. Optimizer zero grad
#         optimizer.zero_grad()

#         # 4. Loss backward
#         loss.backward()

#         # 5. Optimizer step
#         optimizer.step()

#         # Calculate accuracy
#         y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
#         train_acc += accuracy_fn(y_pred_class, y)

#     # Adjust metrics to get average loss and accuracy per batch
#     train_loss = train_loss / len(data_loader)
#     train_acc = train_acc / len(data_loader)

#     print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

#     return train_loss, train_acc

# Function to train the model
def train_model(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs=10, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.to(device)
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                          mode='min',
                                                          factor=0.1,
                                                          patience=3,
                                                          verbose=True)

    for epoch in tqdm(range(epochs)):
        # Training
        model.train()
        train_loss, train_acc = 0, 0

        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)

            # Forward pass
            y_pred = model(X)

            # Calculate loss and accumulate
            loss = loss_fn(y_pred, y)
            train_loss += loss.item()

            # Calculate accuracy and accumulate
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            train_acc += (y_pred_class == y).sum().item()/len(y_pred)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Adjust per batch
        train_loss = train_loss / len(train_dataloader)
        train_acc = train_acc / len(train_dataloader)

        # Testing
        model.eval()
        test_loss, test_acc = 0, 0

        with torch.no_grad():
            for X, y in test_dataloader:
                X, y = X.to(device), y.to(device)

                # Forward pass
                test_pred = model(X)

                # Calculate loss and accumulate
                loss = loss_fn(test_pred, y)
                test_loss += loss.item()

                # Calculate accuracy and accumulate
                test_pred_class = torch.argmax(torch.softmax(test_pred, dim=1), dim=1)
                test_acc += (test_pred_class == y).sum().item()/len(test_pred)

        # Adjust per batch
        test_loss = test_loss / len(test_dataloader)
        test_acc = test_acc / len(test_dataloader)

        # Update scheduler
        scheduler.step(test_loss)

        # Store results
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # Print progress
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

    return results

In [ ]:
model_3 = ImprovedFashionMNISTModel(input_shape=1, hidden_units=64, output_shape=10)
optimizer = torch.optim.Adam(model_3.parameters(), lr=0.001, weight_decay=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
train_time_start_model_3 = timer()

model_3_results = train_model(model_3, train_dataloader, test_dataloader, optimizer, loss_fn, epochs=3)

train_time_end_model_3 = timer()

total_train_time_model_3 = print_train_time(
    start=train_time_start_on_gpu,
    end=train_time_end_on_gpu,
    device=device # Get the actual device the model is on
)

In [ ]:
import pandas as pd
model_3_results = eval_model(
    model=model_3, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_fn=accuracy_fn, device=device
)
compare_results = pd.DataFrame([model_0_results, model_1_results, model_2_results, model_3_results])
compare_results

In [ ]:
# Add training time to results comparison
compare_results["training_time"] = [total_train_time_model_0, total_train_time_model_1, total_train_time_model_2, total_train_time_model_3]
compare_results

In [ ]:
compare_results.set_index("model_name")["model_acc"].plot(kind="barh")
plt.xlabel("accuracy / %")
plt.ylabel("model")

## 9. Make and evaluate random predictions with the best model


In [ ]:
def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):
  pred_probs = []
  model.to(device)
  model.eval()
  with torch.inference_mode():
    for sample in data:
      # Prepare sample (add a batch dimension and pass to target device)
      sample = torch.unsqueeze(sample, dim = 0).to(device)

      # Forward pass (model outputs raw logits)
      pred_logit = model(sample)

      # Get prediction probability (logit to prediction probability)
      pred_prob = torch.softmax(pred_logit.squeeze(), dim = 0)

      # Get pred_prob off GPU and on CPU for further calculations
      pred_probs.append(pred_prob.cpu())

  # Stack the pred_probs to turn list into tensor

  return torch.stack(pred_probs)



In [ ]:
import random
# random.seed(42)
test_samples = []
test_labels = []

for sample, label in random.sample(list(test_data), k=9):
  test_samples.append(sample)
  test_labels.append(label)

# View the first sample shape
test_samples[0].shape


In [ ]:
plt.imshow(test_samples[0].squeeze(), cmap="gray")
plt.title(class_names[test_labels[0]])

In [ ]:
# Make predictions
pred_probs = make_predictions(model=model_3, data=test_samples)

# View first two prediction probabilities
pred_probs[:2]

In [ ]:
# How do we convert prediction_probs into pred labels to compare with test_labels

pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
# Plot predictions
plt.figure(figsize=(9,9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_samples):
  # Create subplot
  plt.subplot(nrows, ncols, i+1)

  #plot target image
  plt.imshow(sample.squeeze(), cmap="gray")

  # fi|nd pred labels in text form
  pred_label = class_names[pred_classes[i]]

  # get truth label
  truth_label = class_names[test_labels[i]]

  # create title
  title_text = f" Pred: {pred_label} | Truth: {truth_label}"

  # check equality
  if pred_label == truth_label:
    plt.title(title_text, fontsize=10, c="g")
  else:
    plt.title(title_text, fontsize=10, c="r")

  plt.axis(False)

## 10. Making a confusion matrix for further prediction evaluation


A confusion matric is a great way to evaluate your classification model visually.

1. Make predictions with our trained model with the test dataset
2. Make confusion matrix `torchmetrics.ConfusionMatrix`
3. Plot the confusion matrix using `mlxtend.plotting.plot_confusion_matrix`




In [ ]:
# 1. Make predictions with trained model

y_preds = []
model_3.eval()
with torch.inference_mode():
  for X, y in tqdm(test_dataloader, desc="Making predictions..."):
    # send data and targets to target device
    X, y = X.to(device), y.to(device)

    #Do forward pass
    y_logit = model_3(X)

    #Turn predictions from logits to prediction probs -> prediction labels
    y_pred = torch.softmax(y_logit.squeeze(), dim=0).argmax(dim=1)
    # Put predictions on CPU for evaluation
    y_preds.append(y_pred.cpu())


# concac list of preds into tensor
# print(y_preds)
y_pred_tensor = torch.cat(y_preds)
y_pred_tensor


In [ ]:
try:
  import torchmetrics, mlxtend
  print(f"mlxtend version: {mlxtend.__version__}")
  assert int(mlxtend.__version__.split(".")[1] >= 19, "mlxtend version should be 0.19.0 or higher")
except:
  !pip install torchmetrics -U mlxtend
  import torchmetrics, mlxtend
  print(f"mlxtend version: {mlxtend.__version__}")


In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. Set up confusion instance and compare predictions to targets
confmat = ConfusionMatrix(num_classes = len(class_names), task="multiclass")
confmat_tensor = confmat(preds = y_pred_tensor, target=test_data.targets)

# 3.  plot confmat

fig, ax = plot_confusion_matrix(
    conf_mat = confmat_tensor.numpy(), # matplotlib needs numpy
    class_names = class_names,
    figsize=(10, 7)
)



## 11. Save and load our best model


In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "03_pytorch_computer_vision_model_3.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
MODEL_SAVE_PATH

# Savemodel state dict

print(f"Saving model to {MODEL_SAVE_PATH}")
torch.save(obj = model_3.state_dict(), f=MODEL_SAVE_PATH) #state_dict is our model's learned parameters on the dataset (weights, biases etc)


In [ ]:
# Load and create model 3 instance from saves model

torch.manual_seed(42)

loaded_model_3 = ImprovedFashionMNISTModel(input_shape = 1, hidden_units = 64, output_shape = len(class_names))

#load in saved state dict
loaded_model_3.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

# Send model to target device
loaded_model_3.to(device)
